## Benchmark MERRCI with Metaphlan3


### Objective
Our analysis shows that <i>Klebsiella pneumoniae</i> and <i>Escherichia coli</i> are the most abundant species in the <a href="https://www.nature.com/articles/nmicrobiol201624">Gibbson et al.</a> dataset. Such results can be biased, as reference sequences of those species are over-represented in the RefSeq database. To validate the effect of database bias, we compared the relative abundance of those species computed by MERRCI and state-of-the-art metagenomic profiler Metaphlan 3 <a href="https://elifesciences.org/articles/65088">(Beghini et al., eLife, 2021)</a>. The effect of the database bias should be lower in Metaphlan 3, as species are counted with clade-specific marker genes with minimal overlap. 

First, 401 stool samples were downloaded from <a href="https://www.nature.com/articles/nmicrobiol201624">Gibbson et al.</a> dataset:

* `1-download_samples.sh`

Then, for each sample, we computed relative abundance with metaphlan3:

* `3-compute_abundance.sh`

In this notebook, we explore the correlation of MERRCI and Metaphlan3 for the top 3 most abundant species.


In [1]:
import pandas as pd
from scipy.stats import pearsonr


#### Constants
klebsiella_name_merrci = 'Klebsiella pneumoniae#abundance'
klebsiella_name_metaphlan = 's__Klebsiella_pneumoniae'

ecoli_name_merrci = 'Escherichia coli#abundance'
ecoli_name_metaphlan = 's__Escherichia_coli'

staph_name_merrci = 'Staphylococcus epidermidis#abundance'
staph_name_metaphlan = 's__Staphylococcus_epidermidis'

merrci_abundance = './data/PTR_species_filtered_metadata.csv'
metaphlan_abundance = './out_abundance/merged_abundance_subset.csv'

merrci_df = pd.read_csv(merrci_abundance, sep=',')
merrci_df = merrci_df[['sample', ecoli_name_merrci, klebsiella_name_merrci, staph_name_merrci]]
metaphlan_df = pd.read_csv(metaphlan_abundance, sep=',')
metaphlan_df[klebsiella_name_metaphlan] = metaphlan_df[klebsiella_name_metaphlan] /100
metaphlan_df[ecoli_name_metaphlan] = metaphlan_df[ecoli_name_metaphlan] /100
metaphlan_df[staph_name_metaphlan] = metaphlan_df[staph_name_metaphlan] /100

merged_df = merrci_df.merge(metaphlan_df, how='left', on='sample')


#### Compute Correlation
corr_klebsiella, pval = pearsonr(list(merged_df[klebsiella_name_merrci]), list(merged_df[klebsiella_name_metaphlan]))
print("Pearson correlation for Klebsiella pneumoniae: {}; p-value: {}".format(corr_klebsiella, pval))


#### Compute Correlation
corr_ecoli, pval = pearsonr(list(merged_df[ecoli_name_merrci]), list(merged_df[ecoli_name_metaphlan]))
print("Pearson correlation for Escherichia coli: {}; p-value: {}".format(corr_ecoli, pval))

#### Compute Correlation
corr_staph, pval = pearsonr(list(merged_df[staph_name_merrci]), list(merged_df[staph_name_metaphlan]))
print("Pearson correlation for Staphylococcus epidermidis: {}; p-value: {}".format(corr_staph, pval))

Pearson correlation for Klebsiella pneumoniae: 0.7688921014794965; p-value: 1.4452519548478219e-77
Pearson correlation for Escherichia coli: 0.8320989697461082; p-value: 1.2660905931597594e-101
Pearson correlation for Staphylococcus epidermidis: 0.8529038773339754; p-value: 7.433550930682316e-112


### Results
MERRCI and Metaphlan3 approaches had a significant Pearson’s correlation for top 3 most abundant species: Klebsiella pneumoniae (r=0.77, p-value=1e-77), Escherichia coli (r=0.83, p-value=1e-101), and Staphylococcus epidermidis (r=0.85, p-value=1e-111). Therefore, we conclude that bias towards species with over-represented sequences in RefSeq did not have a significant effect for <a href="https://www.nature.com/articles/nmicrobiol201624">Gibbson et al.</a> dataset dataset.
